In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv1D, GlobalAveragePooling1D, 
                                     ReLU, Multiply, LSTM, Dense, BatchNormalization, Dropout)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from tabulate import tabulate
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Load CSV
data_path = 'plant_health_datawithnomalization.csv'
data = pd.read_csv(data_path)

# Shuffle data
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Features and Target
X = data[['Soil_Moisture', 'Ambient_Temperature', 'Soil_Temperature', 'Humidity',
          'Light_Intensity', 'Soil_pH', 'Nitrogen_Level', 'Phosphorus_Level',
          'Potassium_Level', 'Chlorophyll_Content', 'Electrochemical_Signal']].values
y = data['Plant_Health_Status'].values

# Reshape X ให้เหมาะกับ Conv1D
X = X.reshape((X.shape[0], X.shape[1], 1))

# Train-Test Split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3333, random_state=42)

In [ ]:
def create_and_train_model(learning_rate):
    # Define the model
    input_seq = Input(shape=(X.shape[1], 1))

    # Left side: Conv_1 → ReLU_1 → Conv_2 → ReLU_2
    conv_1 = Conv1D(filters=128, kernel_size=3, padding='same')(input_seq)
    conv_1 = BatchNormalization()(conv_1)  # Add Batch Normalization
    relu_1 = ReLU()(conv_1)
    dropout_1 = Dropout(0.2)(relu_1)  

    conv_2 = Conv1D(filters=128, kernel_size=3, padding='same')(dropout_1)
    conv_2 = BatchNormalization()(conv_2)  # Add Batch Normalization
    relu_2 = ReLU()(conv_2)
    dropout_2 = Dropout(0.2)(relu_2)  

    # Right side: Conv_1 → Global Pooling → Fc_2 → ReLU_3 → Fc_3 → Sigmoid
    global_pool = GlobalAveragePooling1D()(conv_1)
    fc_2 = Dense(128)(global_pool)
    relu_3 = ReLU()(fc_2)
    dropout_3 = Dropout(0.2)(relu_3)
    fc_3 = Dense(128)(dropout_3)
    sigmoid = Dense(128, activation='sigmoid')(fc_3)

    # Multiply both sides
    multiply = Multiply()([dropout_2, sigmoid])

    # LSTM × 6
    lstm = LSTM(256, return_sequences=True)(multiply)
    for _ in range(4):  
        lstm = LSTM(256, return_sequences=True)(lstm)
    lstm = LSTM(256, return_sequences=False)(lstm)  

    # Fully Connected Layer
    fc = Dense(128, activation='relu')(lstm)
    dropout_4 = Dropout(0.2)(fc)  

    # Regression Output
    output = Dense(1)(dropout_4)

    # Create model
    model = Model(inputs=input_seq, outputs=output)

    # Define optimizer with learning rate
    optimizer = Adam(learning_rate=learning_rate)

    # Compile model
    model.compile(optimizer=optimizer, loss='mse')

    return model

In [ ]:
def train_model(model, learning_rate):
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=20, 
        restore_best_weights=True
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=10,
        min_lr=1e-6
    )

    history = model.fit(
        X_train, y_train,
        epochs=200,
        batch_size=8,  # Increased batch size
        validation_data=(X_val, y_val),
        verbose=1,
        callbacks=[early_stopping, reduce_lr]
    )

    plt.figure(figsize=(8, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss (MSE)')
    plt.title(f'Learning Curve (LR={learning_rate})')
    plt.legend()
    plt.grid()
    plt.show()

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    metrics = [
        learning_rate,
        r2_score(y_test, y_pred_test), mean_absolute_error(y_test, y_pred_test),
        np.mean(y_test - y_pred_test), np.sqrt(mean_squared_error(y_test, y_pred_test)),
        r2_score(y_train, y_pred_train), mean_absolute_error(y_train, y_pred_train),
        np.mean(y_train - y_pred_train), np.sqrt(mean_squared_error(y_train, y_pred_train))
    ]
    return metrics

learning_rates = [0.001]
results = []
for lr in learning_rates:
    model = create_and_train_model(lr)
    results.append(train_model(model, lr))

headers = ["Learning Rate", "Test R²", "Test MAE", "Test MBE", "Test RMSE",
           "Train R²", "Train MAE", "Train MBE", "Train RMSE"]
print(tabulate(results, headers=headers, tablefmt="grid"))

df_results = pd.DataFrame(results, columns=headers)
